In [ ]:
import numpy as np
import pandas as pd
import gc

In [ ]:
sub_0 = pd.read_csv('../input/1023folds-lilkoke/1023fold0.csv')
sub_1 = pd.read_csv('../input/1023folds-lilkoke/1023fold1.csv')
sub_2 = pd.read_csv('../input/1023folds-lilkoke/1023fold2.csv')
sub_3 = pd.read_csv('../input/gb-vpp-pulp-fiction/median_submission.csv')
sub_4 = pd.read_csv('../input/new-ensemble-of-public-notebooks/submission.csv')
sub_5 = pd.read_csv('../input/vpp-a-basic-ensembling-technique/submission_pp.csv')
sub_6 = pd.read_csv('../input/ensemble-without-overfitting-risk/submission_median.csv')


train_df = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test_df = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
submission = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")
unique_pressures = train_df["pressure"].unique()
sorted_pressures = np.sort(unique_pressures)
total_pressures_len = len(sorted_pressures)

In [ ]:
def find_nearest(prediction, threshold=0.25):
    insert_idx = np.searchsorted(sorted_pressures, prediction)
    if insert_idx == total_pressures_len:
        # If the predicted value is bigger than the highest pressure in the train dataset,
        # return the max value.
        return sorted_pressures[-1]
    elif insert_idx == 0:
        # Same control but for the lower bound.
        return sorted_pressures[0]
    lower_val = sorted_pressures[insert_idx - 1]
    upper_val = sorted_pressures[insert_idx]
    #print(lower_val, upper_val)
    
    if prediction <= lower_val + P_STEP*threshold :
        return lower_val
    elif prediction >= upper_val - P_STEP*threshold:
        return upper_val
    else:
        return prediction

In [ ]:
targets = train_df[['pressure']].to_numpy().reshape(-1, 80)
pressure = targets.squeeze().reshape(-1,1).astype('float32')

P_MIN = np.min(pressure)
P_MAX = np.max(pressure)
P_STEP = (pressure[1] - pressure[0])[0]
print('Min pressure: {}'.format(P_MIN))
print('Max pressure: {}'.format(P_MAX))
print('Pressure step: {}'.format(P_STEP))
print('Unique values:  {}'.format(np.unique(pressure).shape[0]))

del pressure
gc.collect()

In [ ]:
submission.head()

In [ ]:
submission = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")
frames = [sub_0.pressure.rename("sub0"), sub_1.pressure.rename("sub1"), sub_2.pressure.rename("sub2"), sub_3.pressure.rename("sub3"), sub_4.pressure.rename("sub4"), sub_5.pressure.rename("sub5"), sub_6.pressure.rename("sub6")]
concat = pd.concat(frames, axis = 1)
submission["pressure"] = concat.median(axis = 1)
concat.head()

In [ ]:
submission["pressure"].apply(find_nearest)
submission["pressure"] = np.clip(submission.pressure, P_MIN, P_MAX)
submission.to_csv('submission.csv', index=False)
submission.head(5)